<a href="https://colab.research.google.com/github/kt13581358/ai_3/blob/main/exe3(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-huggingface langchain_community faiss-cpu docx2txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [2]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
model_name = "HooshvareLab/bert-base-parsbert-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [3]:
text = "افراد و آگاهی، همه دانش صحیح انتقال با هوشواره معتقدیم در ما"
embed = hf_embedding.embed_query(text)
print(f"طول embedding: {len(embed)}")

طول embedding: 768


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import Docx2txtLoader

In [9]:
files = [
    {"path": "/content/esfahan.docx", "source": "esfahan"},
    {"path": "/content/qhazvin.docx", "source": "qhazvin"},
    {"path": "/content/shiraz.docx", "source": "shiraz"}
]

all_chunks = []

for file in files:

  loader = Docx2txtLoader(file["path"])
  documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=500,
      chunk_overlap=50
  )
  chunks = text_splitter.split_documents(documents)

  for chunk in chunks:
      chunk.metadata["source"] = file["source"]
      all_chunks.append(chunk)

  print(f"فایل {file['path']} با موفقیت لود شد. تعداد تکه‌ها: {len(chunks)}")



فایل /content/esfahan.docx با موفقیت لود شد. تعداد تکه‌ها: 25
فایل /content/qhazvin.docx با موفقیت لود شد. تعداد تکه‌ها: 33
فایل /content/shiraz.docx با موفقیت لود شد. تعداد تکه‌ها: 24


In [10]:
from langchain_community.vectorstores import FAISS
import faiss

In [12]:
index = faiss.IndexFlatL2(len(hf_embedding.embed_query("سلام")))
vector_store = FAISS.from_documents(
    documents=all_chunks,
    embedding=hf_embedding
)

query = "غذاهای سنتی چیست؟"
results = vector_store.similarity_search(
    query,
    k=2,
    filter={"source": "qhazvin"}
)

print("\nنتایج جستجوی معنایی با فیلتر (منبع: qhazvin):")
for i, res in enumerate(results, 1):
    print(f"* نتیجه {i}: {res.page_content} [{res.metadata}]")


نتایج جستجوی معنایی با فیلتر (منبع: qhazvin):
* نتیجه 1: سفال و سرامیک

محصولات سفال و سرامیک نقش بسیار مهمی در صنایع‌دستی قزوین دارند. آثار سفالینه و سرامیکی با طرح‌ها و نقوش متنوع از جمله سوغاتی‌های محبوب این شهر هستند.

شیرینی‌های محلی

شیرینی‌های محلی مانند شیرینی قندی، پادرازی، نان چای و نان‌بادامی قزوین از جمله سوغاتی‌های خوشمزه و محبوب قزوین هستند.

عطر و ادکلن [{'source': 'qhazvin'}]
* نتیجه 2: شیرین‌پلو

شیرین پلو یکی از غذاهای سنتی و محبوب استان قزوین است که با طعم شیرین و دلچسب خود، طرفداران زیادی دارد. این غذا از ترکیب برنج، خلال پرتقال، شربت، خلال بادام، خلال پسته، زعفران، کره، سیب زمینی و سینه مرغ تهیه می‌شود.

 آش دوغ قزوین [{'source': 'qhazvin'}]
